Exploring Basics of Regular Expressions

In [30]:
# Required Modules

import re
import pandas as pd
import numpy as np
import regex
import pygrep
import flashtext
import spacy
import loguru
import pyspark
import kafka
import os
import gzip
import shutil
import requests

In [12]:
# File Paths

src_folder_path = "C://Users//mquay//Documents//GitHub//Personal//nlp-practice//src"
notebooks_path = "C://Users//mquay//Documents//GitHub//Personal//nlp-practice//src//notebooks"
scripts_path = "C://Users//mquay//Documents//GitHub//Personal//nlp-practice//src//scripts"
datasets_path = "C://Users//mquay//Documents//GitHub//Personal//nlp-practice//datasets"
csvs_path = "C://Users//mquay//Documents//GitHub//Personal//nlp-practice//datasets//csv"
jsons_path = "C://Users//mquay//Documents//GitHub//Personal//nlp-practice//datasets//json"
txt_path = "C://Users//mquay//Documents//GitHub//Personal//nlp-practice//datasets//txt"

In [ ]:
# Important Functions



In [32]:
# Loading File

robotstxt_file = os.path.join(txt_path, "robotstxt.paths.gz")

In [33]:
# Accessing File

robotstxt_urls = []
with gzip.open(robotstxt_file, "rt", encoding="utf-8") as file:
    for i, line in enumerate(file):
        robotstxt_urls.append(line.strip())

In [34]:
# How many files exist?
len(robotstxt_urls)

90000

In [35]:
# Sampling 10 files for exploration

sample_robotstxt_urls = robotstxt_urls[:10]
len(sample_robotstxt_urls)

10

In [37]:
sample_robotstxt_urls[0]

'crawl-data/CC-MAIN-2025-05/segments/1736703361941.29/robotstxt/CC-MAIN-20250126135402-20250126165402-00000.warc.gz'

## Exploration

In [57]:
text_exploration = sample_robotstxt_urls[0]

In [ ]:
# re.search()

first_num_occ = re.search(r"\d+", text_exploration)
print(first_num_occ.group())

2025


In [61]:
first_word_occ = re.search(r"\w+", text_exploration)
print(first_word_occ.group())

crawl


In [79]:
multiple_occ1 = re.findall(r"main", text_exploration)
print(multiple_occ1)

[]


In [67]:
len(multiple_occ1)

111

In [72]:
multiple_occ2 = re.findall(r"\w[main]", text_exploration)
print(multiple_occ2)

['ra', 'da', 'ta', 'gm', 'en', 'wa']


In [ ]:
# re.findall()



In [47]:
numbers = re.search(r"\d+", sample_robotstxt_urls[0])
print(f"Extracted Numbers {numbers.group()}")

Extracted Numbers 2025


In [36]:
# Downloading 10 files

base_url_robotstxt = "https://data.commoncrawl.org/"

for i in range(len(sample_robotstxt_urls)):
    robotstxt_url = os.path.join(base_url_robotstxt, sample_robotstxt_urls[i])
    response = requests.get(robotstxt_url, stream=True)
    with open(sample_robotstxt_urls[i], "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
            print("Downloaded WARC file successfully!")

FileNotFoundError: [Errno 2] No such file or directory: 'crawl-data/CC-MAIN-2025-05/segments/1736703361941.29/robotstxt/CC-MAIN-20250126135402-20250126165402-00000.warc.gz'

## `re.search()` - Find the first match

In [ ]:


for file in 

In [7]:
text1 = "Order number 12345 was processed on 2024-02-17."

numbers = re.search(r"\d+", text1)
print(f"Extracted Numbers: {numbers.group()}")

Extracted Numbers: 12345


## `re.findall()` - Find all matches

## `re.match()` - Does pattern match at start of string?

## `re.fullmatch()` - Does entire string match pattern?

## `re.sub()` - Replaces occurences of a pattern with another string

## `re.split()` - Splits a string based on regex pattern

## `re.compile()` - Precompiles a regex for repeated use